In [ ]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession
from delta.tables import DeltaTable
from pyspark.sql.functions import col, to_json, lit, collect_list, size, avg

import logging
import os

In [ ]:
# configura os logs
LEVEL = os.getenv("LOG_LEVEL", "INFO").upper()

def setup_logger():
    """
    Configura o logger para o formato desejado.
    """
    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    logger = logging.getLogger(__name__)
    
    return logger

In [ ]:
# configura sessao spark
def sessao_spark(app_name):

    minio_endpoint = "http://minio:9000"
    minio_access_key = "lakehouse"
    minio_secret_key = "2fUDaiyqNFhoMmgYuXjO4d24fchviXQjM2TWTgUe"

    # Lista todos os JARs na pasta
    jars_dir = "./jars"
    jars = [os.path.join(jars_dir, jar) for jar in os.listdir(jars_dir) if jar.endswith(".jar")]
    jars_str = ",".join(jars)
    
    # Configuração do Spark
    spark = (
        SparkSession 
        .builder 
        .appName(f"{app_name}") 
        .master("spark://spark-master:7077") 
        .config("spark.jars", jars_str)
        .config("spark.executor.memory", "3g") 
        .config("spark.executor.cores", "2")
        .config("spark.sql.files.maxPartitionBytes", "134217728")
        .config("spark.hadoop.fs.s3a.endpoint", minio_endpoint) 
        .config("spark.hadoop.fs.s3a.access.key", minio_access_key) 
        .config("spark.hadoop.fs.s3a.secret.key", minio_secret_key) 
        .config("spark.hadoop.fs.s3a.multipart.size", "104857600")
        .config("spark.hadoop.fs.s3a.path.style.access", "true") 
        .config("spark.hadoop.fs.s3a.fast.upload", "true")
        .config("spark.hadoop.fs.s3a.connection.maximum", "100")
        .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 
        .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") 
        .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") 
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") 
        .getOrCreate()
    )

    return spark

In [ ]:
# configs
spark = sessao_spark('dsa-streaming')

logger = setup_logger()
logger.info("Iniciando o script.")

logger.info(spark)
logger.info(f"Configs: {SparkConf().getAll()}")
spark.sparkContext.setLogLevel("ERROR")

minio_bucket = f"s3a://production/landing/dsa"